In [ ]:
! pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-no

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

from tqdm import tqdm

In [ ]:
tagger = SequenceTagger.load("flair/ner-german-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2024-05-23 08:22:16,626 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>


In [ ]:
all_characters = []

In [ ]:
import string

In [ ]:
parts = []

with open('Nibelungenlied_A.txt', 'r', encoding='utf-8') as f:
    current_text = ""

    for line in f.readlines():
        line = line.replace('\xa0', ' ')
        new = line.translate(str.maketrans('', '', string.punctuation))
        current_text += new
        if len(current_text.split('\n')) > 20:
          parts.append(current_text)
          current_text = ""

    if current_text:
      parts.append(current_text)

In [ ]:
for part in tqdm(parts):
  sentence = Sentence(part)
  tagger.predict(sentence)

  for x in sentence.get_spans('ner'):
    if str(x).split()[3] == 'PER':
      prob = str(x).split()[4].replace('(', '')
      prob = prob.replace(')', '')
      prob = float(prob)
      if prob > 0.998:
        all_characters.append(str(x).split()[1].replace('"', ''))

100%|██████████| 709/709 [28:06<00:00,  2.38s/it]


In [ ]:
with open('characters Nieb A.txt', 'a', encoding='utf-8') as f:
  for element in all_characters:
    if 'michel' not in element:
      f.write(element)
      f.write('\n')

Теперь надо найти конструкции (https://ids.clld.org/contributions/193 термины родства взяты отсюда, нормализированая орфография, поэтому пройдёсь и исправлю на нужное)

In [ ]:
kinship_dict = {}

In [ ]:
with open('kinship_mhg.txt', 'r', encoding='utf-8') as f:
  for line in f.readlines():
    word = line.split('\t')[1][:-1]
    if word not in kinship_dict:
      kinship_dict[word] = []
      kinship_dict[word].append(line.split('\t')[0])
    else:
      kinship_dict[word].append(line.split('\t')[0])

In [ ]:
kinship_dict

{'man': ['man', 'woman', 'male', 'husband'],
 'wip': ['woman', 'female', 'wife'],
 'knabe': ['boy'],
 'kneht': ['boy'],
 'maget': ['girl'],
 'dierne': ['girl'],
 'kint': ['child', 'infant, baby'],
 'sugelinc': ['infant, baby'],
 'gemahele': ['husband', 'wife'],
 'kone': ['wife'],
 'vater': ['father'],
 'můter': ['mother'],
 'altern': ['parents'],
 'eltern': ['parents'],
 'eman': ['married man'],
 'ewip': ['married woman'],
 'sun': ['son'],
 'tohter': ['daughter'],
 'nachkome': ['offspring (son or daughter)'],
 'brůder': ['brother'],
 'elterer brůder': ['older brother'],
 'jüngerer brůder': ['younger brother'],
 'swester': ['sister'],
 'eltere swester': ['older sister'],
 'jüngere swester': ['younger sister'],
 'geswisterde': ['sibling'],
 'zwinelinc': ['twins'],
 'zwilinc': ['twins'],
 'ane': ['grandfather', 'grandmother'],
 'eltervater': ['grandfather'],
 'grise': ['old man'],
 'altvater': ['old man'],
 'eltermůter': ['grandmother'],
 'altez wip': ['old woman'],
 'eninkel': ['grands

In [ ]:
text =''

with open('Nibelungenlied_A.txt', 'r', encoding='utf-8') as f:
  for line in f.readlines():
    line = line.replace('\xa0', ' ')
    line = line.replace('   ', ' ')
    line = line.replace('«', '')
    line = line.replace('»', '')
    new = line.translate(str.maketrans('', '', string.punctuation))
    text = text+new

In [ ]:
pronouns = ['der', 'daz', 'diu', 'die', 'den', 'dem', 'des']

In [ ]:
all_characters = []

with open('characters Nieb A.txt', 'r', encoding='utf-8') as f:
  for line in f.readlines():
    all_characters.append(line[:-1])

In [ ]:
fragments = []

In [ ]:
for line in text.split('\n'):
    words = line.split()
    was_in_constr = []
    for i in range(len(words)):
        word = words[i]
        if word in all_characters:
            if i+1 < len(words) and words[i+1] in pronouns:
                #следующее слово - местоимение
                for j in range(i+1, len(words)):
                  if words[j] in kinship_dict:
                    was_in_constr.append(word)
                    fragment = ' '.join(words[i:])
                    fragments.append(fragment)
                    break
                  else:
                    for j in range(max(0, i-3), min(len(words), i+4)):
                      if words[j] in kinship_dict:
                          #персонаж в окрестности термина родства
                          for k in range(max(0, j-3), min(len(words), j+4)):
                              if words[k] in all_characters and words[k] not in was_in_constr:
                                if k > j:
                                  was_in_constr.append(word)
                                  fragment = ' '.join(words[j:k+1])
                                  fragments.append(fragment)
                                  break
                                else:
                                  was_in_constr.append(word)
                                  fragment = ' '.join(words[k:j+1])
                                  fragments.append(fragment)
                                  break
            else:
                #термины родства
                for j in range(max(0, i-3), min(len(words), i+4)):
                    if words[j] in kinship_dict:
                        #персонаж в окрестности термина родства
                        for k in range(max(0, j-3), min(len(words), j+4)):
                            if words[k] in all_characters and words[k] not in was_in_constr:
                              if k > j:
                                was_in_constr.append(word)
                                fragment = ' '.join(words[j:k+1])
                                fragments.append(fragment)
                                break
                              else:
                                was_in_constr.append(word)
                                fragment = ' '.join(words[k:j+1])
                                fragments.append(fragment)
                                break
    #print(was_in_constr)